In [11]:
import cv2
import argparse
import numpy as np
import os
import pickle
from tqdm import tqdm

In [13]:
descpt_id = 'sift'
images_path = '/home/garima/Desktop/Beer_images/distances/database'
images = os.listdir(images_path)
images = [os.path.join(images_path, item) for item in images]

if descpt_id == 'sift':
    feature_extractor = cv2.xfeatures2d.SIFT_create()
elif descpt_id == 'surf':
    feature_extractor = cv2.xfeatures2d.SURF_create(extended=True)   # For getting descriptor of size 128 instead of 64
else:
    feature_extractor = cv2.ORB_create()
print('[INFO] Creating descriptors...')
descriptors = dict()
for image in tqdm(images, desc='Processing images: '):
    image_gray = cv2.imread(image, 0)
    image_gray = cv2.resize(image_gray,(400,300))
    try:
        keypoint, descriptor = feature_extractor.detectAndCompute(image_gray, None)
    except Exception as e:
        print(e)
        pdb.set_trace()
    image_name = image.split('/')[-1].split('.')[0]
    if image_name not in descriptors.keys():
        descriptors[image_name] = descriptor
    else:
        print('[ERROR] Issue with file {}!\nCheck for duplicate images.'.format(image))
with open(descriptors1, 'wb+') as file:
    pickle.dump(descriptors, file)
# else:
#     print('[INFO] Using previously saved descriptors...')
#     with open(args.descriptors, 'rb') as file:
#         descriptors = pickle.load(file)
if len(descriptors.keys()) != len(images):
    raise ValueError("Number of descriptors do not match with number of images in the dataset folder. Create a new set of descriptors.")
else:
    print("done")

Processing images:   5%|▌         | 1/20 [00:00<00:02,  8.39it/s]

[INFO] Creating descriptors...


Processing images: 100%|██████████| 20/20 [00:01<00:00,  9.83it/s]

done


In [14]:


l = '..distance_sift.pkl'
load = 'Y'
GOOD_MATCH_THRESH = 10

query_images = '../query/'
database_images = '../database/'
descriptor_file = l
load_descriptor =load

database_descriptors = {}
name_list = []
sift = cv2.xfeatures2d.SIFT_create()
#feature_extractor = cv2.xfeatures2d.SIFT_create()

if load_descriptor == 'Y' or load_descriptor =='y':
    print("Loading Pre Computed Descriptors")
    assert os.path.exists(descriptor_file)
    with open(descriptor_file, 'rb') as file:
        database_descriptors = pickle.load(file)
else:
    for image_name in tqdm(os.listdir(database_images)):
        img = cv2.imread(database_images+image_name,0)
        keypoint, descriptor = sift.detectAndCompute(img, None)
        img_name,img_extension = os.path.splitext(image_name)
        database_descriptors[img_name] = descriptor
        name_list.append(img_name)
    with open(descriptor_file, 'wb') as file:
        pickle.dump(database_descriptors, file)

print("--Database Descriptors Loaded--")

correct_count = 0


for query_name in tqdm(os.listdir(query_images)):
    max_count_name = None
    max_count = 0
    query = cv2.imread(query_images+query_name, 0)
    _, q_desc = sift.detectAndCompute(query, None)
    for database_name in database_descriptors.keys():
        d_desc = database_descriptors[database_name]
        FLANN_INDEX_KDTREE = 0
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks = 100)
        flann = cv2.FlannBasedMatcher(index_params, search_params)
        matches = flann.knnMatch(q_desc, d_desc, k=2)
        matches_mask = [[0, 0] for i in range(len(matches))]
        distances = list()
        counts = list()
        for i, (m, n) in enumerate(matches):
            if m.distance < 0.7 * n.distance:
                matches_mask[i] = [1, 0]
                distances.append(m.distance)
        counts.append(len(distances))
        if len(distances) != 0:
            if np.max(counts) > max_count:
                max_count = np.max(counts)
                max_count_name = database_name.split('/')[-1].split('.')[0]
    if query_name.split('/')[-1].split('.')[0] == max_count_name:
        correct_count += 1
    else:
        print('[ERROR] database_image: {}, query_image: {}'.format(max_count_name, query_name.split('/')[-1].split('.')[0]))
print('[INFO] Correct matches: {} total queries: {} Accuracy: {}'.format(correct_count, len(os.listdir(query_images)), correct_count/len(os.listdir(query_images))))


  0%|          | 0/20 [00:00<?, ?it/s]

Loading Pre Computed Descriptors
--Database Descriptors Loaded--


 10%|█         | 2/20 [00:01<00:11,  1.62it/s]

[ERROR] database_image: miller_lite2, query_image: storm


 65%|██████▌   | 13/20 [00:12<00:08,  1.17s/it]

...

[ERROR] database_image: bira_blonde, query_image: heineken


 95%|█████████▌| 19/20 [00:19<00:01,  1.01s/it]

[ERROR] database_image: corona_extra, query_image: kingfisher


100%|██████████| 20/20 [00:19<00:00,  1.06it/s]

[INFO] Correct matches: 15 total queries: 20 Accuracy: 0.75
